In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle

In [2]:
# Camera calibration

# Counters for storing inside corners of calibration image
noOfCorners_X = 9
noOfCorners_Y = 6
objpoints = []
imgpoints = []
imgs = []
grays = []

objp = np.zeros((noOfCorners_X*noOfCorners_Y, 3), np.float32)
objp[:,:2] = np.mgrid[0:noOfCorners_X, 0:noOfCorners_Y].T.reshape(-1, 2)

#Iterate over calibration images to find corners (images which are not 9x6 are removed from the dataset manually)
#calibration6.jpg to calibration20.jpg
path = [img_path for img_path in glob.glob("./camera_cal/*.jpg")]
for index, filename in enumerate(path):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (noOfCorners_X, noOfCorners_Y), None)
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
        imgs.append(img)
        grays.append(gray)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

#Save the Calibration parameters for future use
cal_pickle = {}
cal_pickle["ret"] = ret
cal_pickle["mtx"] = mtx
cal_pickle["dist"] = dist
cal_pickle["rvecs"] = rvecs
cal_pickle["tvecs"] = tvecs
pickle.dump(cal_pickle, open("camera_cal\cal_pickle.p", "wb"))
print("Done")

Done


In [3]:
#Reading calibration variobales from file
cal_pickle = pickle.load(open("camera_cal\cal_pickle.p", "rb"))
mtx = cal_pickle["mtx"]
dist = cal_pickle["dist"]
print("Done")

Done


In [6]:
#Image Manipulation helper functions
def undistort(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    #ret, corners = cv2.findChessboardCorners(gray, (4, 4), None)
    return undist #, ret, corners
    
def warp(img, ret): #, corners, nx, ny):
    offset = 300
    if ret == True:
        img_size = (img.shape[1], img.shape[0])
        #1280, 720
        
        #src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        #src = np.float32(
        #    [[850,320],
        #     [865,450],
        #     [533,350],
        #     [535,210]   
        #    ])
    
        src = np.float32(
            [[585,440],
             [700,440],
             [1160,720],
             [160,720]
            ])
    #diff 125, 1000
    
        #offset = 100
        #dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
        #                             [img_size[0]-offset, img_size[1]-offset], 
        #                             [offset, img_size[1]-offset]])
        #dst = np.float32(
        #    [[870,240],
        #     [870,370],
        #     [520,370],
        #     [520,240]   
        #    ])
        
        dst = np.float32(
            [[offset,0],
             [img_size[0] - offset,0],
             [img_size[0] - offset,img_size[1]],
             [offset,img_size[1]]
        ])
        
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
    
        warped = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)
        return warped, M, Minv
    
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = img
    
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    if orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    
    abs_sobel = np.absolute(sobel)
    sobel_scaled = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary = np.zeros_like(sobel_scaled)
    binary[(sobel_scaled >= thresh[0]) & (sobel_scaled <= thresh[1])] = 1
    
    return binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = img
    
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    
    combined_gradient = np.sqrt(sobel_x**2 + sobel_y**2)
    scale_factor = np.max(combined_gradient)/255
    combined_gradient = (combined_gradient/scale_factor).astype(np.uint8)
    
    binary = np.zeros_like(combined_gradient)
    binary[(combined_gradient >= mag_thresh[0]) & (combined_gradient <= mag_thresh[1])] = 1
    
    return binary

def dir_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = img
    
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)
    
    dir_gradient = np.arctan2(abs_sobel_y, abs_sobel_x)
       
    binary = np.zeros_like(dir_gradient)
    binary[(dir_gradient >= thresh[0]) & (dir_gradient <= thresh[1])] = 1
    
    return binary

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def slidingWindowSearchByConvolution(warped, img, window_width=50, window_height=80, margin=100):
    window_centroids = []  # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))
        
     # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        #warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        #output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results   
        
        
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    
    return l_points, r_points, template


left_fit_initial = []
right_fit_initial = []
numberOfFramesProcessed = 0
def SlidingWindowByHistogram(binary_warped, undist, Minv, shape1, shape0):
    global left_fit_initial
    global right_fit_initial
    global numberOfFramesProcessed
    
    #print(numberOfFramesProcessed)
    # Set the width of the windows +/- margin
    margin = 100        
    
    
    if numberOfFramesProcessed == 0:
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
        #l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
        
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
            
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        # Set height of windows
        nwindows = 9
        window_height = np.int(binary_warped.shape[0]/nwindows)        
        
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        
        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            
            # Draw the windows on the visualization image
            #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        #Store the current frame fits to future use
        left_fit_initial = left_fit
        right_fit_initial = right_fit
        
    else:
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        left_fit = left_fit_initial
        right_fit = right_fit_initial   
        
        #Check this
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - 100)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + 100))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - 100)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + 100)))  

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 
        
        
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        #Store the current frame fits to future use
        left_fit_initial = left_fit
        right_fit_initial = right_fit
        
        #fit_error = 0.01
        #left_error = ((left_fit_initial[0] - left_fit[0]) ** 2).mean(axis=None)      
        #right_error = ((right_fit_initial[0] - right_fit[0]) ** 2).mean(axis=None)        
        #if left_error < fit_error:
        #    left_fit_initial = 0.75 * left_fit_initial + 0.25 * left_fit   
        #if right_error < fit_error:
        #    right_fit_initial = 0.75 * right_fit_initial + 0.25 * right_fit
        
        #left_fit = left_fit_initial
        #right_fit = right_fit_initial
    
    #left_curvature, right_curvature = radiusOfCurvature(left_lane_inds, right_lane_inds)
    
    #Comment this line only if you are processing test images
    numberOfFramesProcessed = numberOfFramesProcessed + 1
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (shape1, shape0)) 

    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    #plt.imshow(result)
    #plot.inshow()
    
    return result #, left_curvature, right_curvature
        

def radiusOfCurvature(leftx, rightx):
    ploty = np.linspace(0, 719, num=720)
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    return left_curverad, right_curverad
    
def DrawLaneMappings(warped, undist, Minv, shape1, shape0, left_fitx , right_fitx):
    ploty = np.linspace(0, 719, num=720)
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (shape1, shape0)) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    #plt.imshow(result)
    #plt.show()
    return result
    


left_fit = []
right_fit = []
def process_image(img):
    #global left_fit
    #global right_fit
    
    #Undistort the image
    undist = undistort(img)
    
    #Perspective Transform
    warped, M, Minv = warp(undist, True) #, [], 4, 4)
    #warped = undist
    
    #Change Color space to HLS for better lane extraction (S Channel)
    hls = cv2.cvtColor(warped, cv2.COLOR_RGB2HLS)
           
    # Exclude Green
    green_low = np.uint8([0, 210, 0])
    green_high = np.uint8([255, 255, 255])
    green_mask = cv2.inRange(hls, green_low, green_high)
    
    # Include Blue
    blue_low = np.uint8([18, 0, 100])
    blue_high = np.uint8([30, 220, 255])
    blue_mask = cv2.inRange(hls, blue_low, blue_high)
    
    # Apply Both Masks
    mask = cv2.bitwise_or(green_mask, blue_mask)
    masked_hsl = cv2.bitwise_and(hls, hls, mask=mask)
    
    gray = cv2.cvtColor(warped, cv2.COLOR_RGB2GRAY)
    hsl_S = hls[:,:,2]
    #hsl_S = np.zeros_like(hls[:,:,2])
    #hsl_S[hls[:,:,2] >= 80] = 1
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = hsl_S
    
    
    hsl_masked_gray = cv2.cvtColor(masked_hsl, cv2.COLOR_RGB2GRAY)
    hsl_binary = np.zeros_like(hsl_masked_gray)
    hsl_binary[hsl_masked_gray >= 50] = 1
    
    
    #gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=7, thresh=(30, 255))
    #grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=3, thresh=(30, 255))
    mag_binary = mag_thresh(gray, sobel_kernel=7, mag_thresh=(30, 255))
    dir_binary = dir_thresh(gray, sobel_kernel=15, thresh=(0.7, 1.3))                  
    
    combined = np.zeros_like(dir_binary)
    #This line is added to make combined int. Check for side-effects of this
    combined = (combined).astype(np.uint8)
    
   
    #combined[((gradx == 1) | (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    #combined[((gradx == 1) & (hsl_binary == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    #combined[((gradx == 1) & (hsl_binary == 1)) | (mag_binary == 1)] = 1
    combined[((mag_binary == 1) & (dir_binary == 1)) | (hsl_binary == 1)] = 1
    
    
    #Perspective Transform
    #binary_warped, M, Minv = warp(combined, True, [], 4, 4)
    #binary_warped = combined          
    
    #print(combined.shape)
    #l_points, r_points, template = slidingWindowSearchByConvolution(combined, img, 50, 80, 100)
    result = SlidingWindowByHistogram(combined, undist, Minv, img.shape[1], img.shape[0])
        
    #output = cv2.addWeighted(warped, 1, template, 0.5, 0.0) # overlay the orignal road image with window results  
    #unwarped_img = cv2.warpPerspective(output, Minv, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    #unwarped_template = cv2.warpPerspective(template, Minv, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    #output = cv2.addWeighted(img, 1, unwarped_template, 0.5, 0.0) # overlay the orignal road image with window results  
    #result = DrawLaneMappings(template, undist, Minv, img.shape[1], img.shape[0], l_points, r_points)
    
    #radiusOfCurvature(l_points, r_points)

    #plot here when testing
    
    
    return result
    
    #gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ##Blur the image to reduce sharp edges
    #imshape = image.shape
    #kernel_size = 3
    #blur_gray = gaussian_blur(gray, kernel_size)
    #edges = canny(blur_gray, 50, 250)
    
    #plt.imshow(edges, cmap="gray")
    #plt.show()

In [7]:

#Reading a frame from the project video
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'project_video_output.mp4'
#clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

#path = [img_path for img_path in glob.glob("./test_images/test*")]
#for index, filename in enumerate(path):
#    img = mpimg.imread(filename)
#    out = process_image(img)


[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [05:03<00:00,  4.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 5min 4s
